In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import os
from keras.preprocessing.image import  ImageDataGenerator
from keras.layers import Dense,Flatten,GlobalAveragePooling2D,Input,Conv2D,MaxPooling2D,Dropout
from keras.models import Model,load_model,Sequential
from keras.callbacks import EarlyStopping

In [ ]:
#generate training and test images
TARGET_SIZE=(224,224)
INPUT_SIZE=(224,224,3)
BATCHSIZE=32	#could try 128 or 32

#Normalization
train_datagen = ImageDataGenerator(rescale=1./255)

test_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
        '/content/drive/MyDrive/Transfer_IDS_IIoT/Datasets/train_A',
        target_size=TARGET_SIZE,
        batch_size=BATCHSIZE,
        class_mode='categorical')
validation_generator = test_datagen.flow_from_directory(
        '/content/drive/MyDrive/Transfer_IDS_IIoT/Datasets/test_A',
        target_size=TARGET_SIZE,
        batch_size=BATCHSIZE,
        class_mode='categorical')

Found 5380 images belonging to 15 classes.
Found 1352 images belonging to 15 classes.


In [ ]:
input_shape=INPUT_SIZE
num_class=15


In [ ]:
#Hyperparameters
# f1,f2,f3 : Filter Sizes
# k : kernel_size
# a1, a2: activation functions
# d1: drop out
# op: optimizer
# ep: number of epochs

def CNN_model( f1, f2, f3, k, a1, a2, d1, op, ep):
  model = Sequential()
  model.add(Conv2D(filters = f1, kernel_size = (k, k), strides=(1,1), input_shape=input_shape,padding='same',activation=a1))
  model.add(Conv2D(filters = f1, kernel_size = (k, k), strides=(1,1), padding='same',activation=a1))
  model.add(MaxPooling2D(pool_size=(2,2)))
  model.add(Conv2D(filters = f2, kernel_size = (k, k), strides=(1,1),padding='same',activation = a2))
  model.add(Conv2D(filters = f2, kernel_size = (k, k), strides=(1,1),padding='same',activation = a2))
  model.add(GlobalAveragePooling2D())
  model.add(Dense(units = f3, activation = a2))
  model.add(Dropout(rate=d1))
  model.add(Dense(num_class,activation='softmax'))
  model.compile(loss='categorical_crossentropy',optimizer=op,metrics=['accuracy'])
  es = EarlyStopping(monitor="val_accuracy", patience = 5)
  model.fit(train_generator,steps_per_epoch=len(train_generator),epochs=ep,validation_data=validation_generator,
            validation_steps=len(validation_generator), callbacks=[es], verbose=0)
  return model

In [ ]:
from random import choice
from random import uniform
from numpy.random import randint

In [ ]:
def initialization_cnn():
  parameters = {}
  f1 = choice([32, 64])
  parameters["f1"] = f1
  f2 = choice([64, 128])
  parameters["f2"] = f2
  f3 = choice([128, 256])
  parameters["f3"] = f3
  k = choice([3,5])
  parameters["k"] = k
  a1 = choice(["relu", "selu", "elu"])
  parameters["a1"] = a1
  a2 = choice(["relu", "selu", "elu"])
  parameters["a2"] = a2
  d1 = round(uniform(0.1, 0.6), 1)
  parameters["d1"] = d1
  op = choice(["adam", "nadam", "adamax", "adagrad"])
  parameters["op"] = op
  ep = randint(10, 25)
  parameters["ep"] = ep
  return parameters

In [ ]:
def generate_population_cnn(n):
  population = []
  for i in range(n):
    chromosome = initialization_cnn()
    population.append(chromosome)
  return population

In [ ]:
# Fitness evaluation metric: Classification Accuracy
def fitness_evaluation(model):
  metrics = model.evaluate(validation_generator)
  return metrics[1]

In [ ]:
# Roulette wheel selection method
def selection(population_fitness):
  total = sum(population_fitness)
  percentage = [round((x/total) * 100) for x in population_fitness]
  selection_wheel = []
  for pop_index,num in enumerate(percentage):
    selection_wheel.extend([pop_index]*num)
  parent1_ind = choice(selection_wheel)
  parent2_ind = choice(selection_wheel)
  return [parent1_ind, parent2_ind]

In [ ]:
def crossover_cnn(parent1, parent2):
  child1 = {}
  child2 = {}

  child1["f1"] = choice([parent1["f1"], parent2["f1"]])
  child1["f2"] = choice([parent1["f2"], parent2["f2"]])
  child1["f3"] = choice([parent1["f3"], parent2["f3"]])

  child2["f1"] = choice([parent1["f1"], parent2["f1"]])
  child2["f2"] = choice([parent1["f2"], parent2["f2"]])
  child2["f3"] = choice([parent1["f3"], parent2["f3"]])

  child1["k"] = choice([parent1["k"], parent2["k"]])
  child2["k"] = choice([parent1["k"], parent2["k"]])

  child1["a1"] = parent1["a2"]
  child2["a1"] = parent2["a2"]

  child1["a2"] = parent2["a1"]
  child2["a2"] = parent1["a1"]

  child1["d1"] = parent1["d1"]
  child2["d1"] = parent2["d1"]

  child1["op"] = parent2["op"]
  child2["op"] = parent1["op"]

  child1["ep"] = parent1["ep"]
  child2["ep"] = parent2["ep"]
  return [child1, child2]

In [ ]:
def mutation_cnn(chromosome):
  flag = randint(0,40)
  if flag <= 20:
    chromosome["ep"] += randint(0, 10)
  return chromosome

In [ ]:
generations = 5
threshold = 90
num_pop = 5

population = generate_population_cnn(num_pop)

for generation in range(generations):

  population_fitness = []
  for chromosome in population:
    f1 = chromosome["f1"]
    f2 = chromosome["f2"]
    f3 = chromosome["f3"]
    k = chromosome["k"]
    a1 = chromosome["a1"]
    a2 = chromosome["a2"]
    d1 = chromosome["d1"]
    op = chromosome["op"]
    ep = chromosome["ep"]

    try:
      model = CNN_model(f1, f2, f3, k, a1, a2, d1, op, ep)
      acc = fitness_evaluation(model)
      print("Parameters: ", chromosome)
      print("Accuracy: ", round(acc,5))
    except:
      acc=0
      print("Parameters: ", chromosome)
      print("Invalid parameters - Build fail")

    population_fitness.append(acc)

  parents_ind = selection(population_fitness)
  parent1 = population[parents_ind[0]]
  parent2 = population[parents_ind[1]]

  children = crossover_cnn(parent1, parent2)
  child1 = mutation_cnn(children[0])
  child2 = mutation_cnn(children[1])

  population.append(child1)
  population.append(child2)

  print("Generation ", generation+1," Outcome: ")
  if max(population_fitness) >= threshold:
    print("Obtained desired accuracy: ", max(population_fitness))
    break
  else:
    print("Maximum accuracy in generation {} : {}".format(generation+1, max(population_fitness)))

  first_min = min(population_fitness)
  first_min_ind = population_fitness.index(first_min)
  population.remove(population[first_min_ind])
  second_min = min(population_fitness)
  second_min_ind = population_fitness.index(second_min)
  population.remove(population[second_min_ind])

43/43 [==============================] - 6s 133ms/step - loss: 0.0444 - accuracy: 0.9919
Parameters:  {'f1': 32, 'f2': 64, 'f3': 256, 'k': 5, 'a1': 'elu', 'a2': 'elu', 'd1': 0.2, 'op': 'adamax', 'ep': 15}
Accuracy:  0.99186
Parameters:  {'f1': 64, 'f2': 128, 'f3': 256, 'k': 3, 'a1': 'relu', 'a2': 'elu', 'd1': 0.4, 'op': 'adam', 'ep': 20}
Invalid parameters - Build fail


In [ ]:
#      XCEPTION MODEL


from keras.applications.xception import  Xception

In [ ]:
#Hyperparameters
# frozen: Frozen layers
# f1: Number of units in dense layer
# a1: Activation functions
# d1: Drop out
# op: Optimizer
# ep: Number of epochs

#Define Xception Model
def Xception_model(frozen, f1, a1, d1, op, ep):
  model_fine_tune = Xception(include_top=False, weights='imagenet', input_shape=input_shape)
  for layer in model_fine_tune.layers[:frozen]:           #125
      layer.trainable = False
  for layer in model_fine_tune.layers[frozen:]:
      layer.trainable = True
  model = GlobalAveragePooling2D()(model_fine_tune.output)
  model=Dense(units=f1,activation=a1)(model)        #256
  model=Dropout(d1)(model)                             #0.4
  model = Dense(num_class, activation='softmax')(model)
  model = Model(model_fine_tune.input, model, name='xception')
  model.compile(loss='categorical_crossentropy',optimizer=op,metrics=['accuracy'])
  es = EarlyStopping(monitor="val_accuracy", patience = 2)
  model.fit(train_generator,steps_per_epoch=len(train_generator),epochs=ep,validation_data=validation_generator,
            validation_steps=len(validation_generator), callbacks=[es], verbose=0)
  return model

In [ ]:
from random import choice
from random import uniform
from numpy.random import randint

In [ ]:
def initialization_xception():
  parameters = {}
  frozen = choice(range(100, 150, 1))
  parameters["frozen"] = frozen
  f1 = choice([128, 256])
  parameters["f1"] = f1
  a1 = choice(["relu", "selu", "elu"])
  parameters["a1"] = a1
  d1 = round(uniform(0.1, 0.6), 1)
  parameters["d1"] = d1
  op = choice(["adam", "nadam", "adamax", "adagrad"])
  parameters["op"] = op
  ep = randint(10, 25)
  parameters["ep"] = ep
  return parameters

In [ ]:
def generate_population_xception(n):
  population = []
  for i in range(n):
    chromosome = initialization_xception()
    population.append(chromosome)
  return population

In [ ]:
# Fitness evaluation metric: Classification Accuracy
def fitness_evaluation(model):
  metrics = model.evaluate(validation_generator)
  return metrics[1]

In [ ]:
# Roulette wheel selection method
def selection(population_fitness):
  total = sum(population_fitness)
  percentage = [round((x/total) * 100) for x in population_fitness]
  selection_wheel = []
  for pop_index,num in enumerate(percentage):
    selection_wheel.extend([pop_index]*num)
  parent1_ind = choice(selection_wheel)
  parent2_ind = choice(selection_wheel)
  return [parent1_ind, parent2_ind]

In [ ]:
def crossover_xception(parent1, parent2):
  child1 = {}
  child2 = {}

  child1["frozen"] = choice([parent1["frozen"], parent2["frozen"]])
  child2["frozen"] = choice([parent1["frozen"], parent2["frozen"]])

  child1["f1"] = choice([parent1["f1"], parent2["f1"]])
  child2["f1"] = choice([parent1["f1"], parent2["f1"]])

  child1["a1"] = parent2["a1"]
  child2["a1"] = parent1["a1"]

  child1["d1"] = parent1["d1"]
  child2["d1"] = parent2["d1"]

  child1["op"] = parent1["op"]
  child2["op"] = parent2["op"]

  child1["ep"] = parent1["ep"]
  child2["ep"] = parent2["ep"]
  return [child1, child2]

In [ ]:
def mutation_xception(chromosome):
  flag = randint(0,40)
  if flag <= 20:
    chromosome["ep"] += randint(0, 10)
  return chromosome

In [ ]:
generations = 5
threshold = 90
num_pop = 3

population = generate_population_xception(num_pop)

for generation in range(generations):

  population_fitness = []
  for chromosome in population:
    frozen = chromosome["frozen"]
    f1 = chromosome["f1"]
    a1 = chromosome["a1"]
    d1 = chromosome["d1"]
    op = chromosome["op"]
    ep = chromosome["ep"]

    try:
      model = Xception_model(frozen, f1, a1, d1, op, ep)
      acc = fitness_evaluation(model)
      print("Parameters: ", chromosome)
      print("Accuracy: ", round(acc,5))
    except:
      acc=0
      print("Parameters: ", chromosome)
      print("Invalid parameters - Build fail")

    population_fitness.append(acc)

  parents_ind = selection(population_fitness)
  parent1 = population[parents_ind[0]]
  parent2 = population[parents_ind[1]]

  children = crossover_xception(parent1, parent2)
  child1 = mutation_xception(children[0])
  child2 = mutation_xception(children[1])

  population.append(child1)
  population.append(child2)

  print("Generation ", generation+1," Outcome: ")
  if max(population_fitness) >= threshold:
    print("Obtained desired accuracy: ", max(population_fitness))
    break
  else:
    print("Maximum accuracy in generation {} : {}".format(generation+1, max(population_fitness)))

  first_min = min(population_fitness)
  first_min_ind = population_fitness.index(first_min)
  population.remove(population[first_min_ind])
  second_min = min(population_fitness)
  second_min_ind = population_fitness.index(second_min)
  population.remove(population[second_min_ind])

In [ ]:
# VGG 16 MODEL
from keras.applications.vgg16 import VGG16





In [ ]:
#Hyperparameters
# frozen: Frozen layers
# f1: Number of units in dense layer
# a1: Activation functions
# d1: Drop out
# op: Optimizer
# ep: Number of epochs

#Define VGG16 Model
def VGG16_model(frozen, f1, a1, d1, op, ep):
  model_fine_tune = VGG16(include_top=False, weights='imagenet', input_shape=input_shape)
  for layer in model_fine_tune.layers[:frozen]:
      layer.trainable = False
  for layer in model_fine_tune.layers[frozen:]:
      layer.trainable = True
  model = GlobalAveragePooling2D()(model_fine_tune.output)
  model=Dense(units=f1,activation=a1)(model)
  model=Dropout(d1)(model)
  model = Dense(num_class, activation='softmax')(model)
  model = Model(model_fine_tune.input, model, name='vgg')
  model.compile(loss='categorical_crossentropy',optimizer=op,metrics=['accuracy'])
  es = EarlyStopping(monitor="val_accuracy", patience = 4)
  model.fit(train_generator,steps_per_epoch=len(train_generator),epochs=ep,validation_data=validation_generator,
            validation_steps=len(validation_generator), callbacks=[es], verbose=0)
  return model

In [ ]:
from random import choice
from random import uniform
from numpy.random import randint

In [ ]:
def initialization_vgg16():
  parameters = {}
  #frozen = choice([8, 16, 24])
  frozen = choice(range(8, 25, 1))
  parameters["frozen"] = frozen
  f1 = choice([128, 256])
  parameters["f1"] = f1
  a1 = choice(["relu", "selu", "elu"])
  parameters["a1"] = a1
  d1 = round(uniform(0.1, 0.6), 1)
  parameters["d1"] = d1
  op = choice(["adam", "nadam", "adamax", "adagrad"])
  parameters["op"] = op
  ep = randint(10, 25)
  parameters["ep"] = ep
  return parameters

In [ ]:
def generate_population_vgg16(n):
  population = []
  for i in range(n):
    chromosome = initialization_vgg16()
    population.append(chromosome)
  return population

In [ ]:
# Fitness evaluation metric: Classification Accuracy
def fitness_evaluation(model):
  metrics = model.evaluate(validation_generator)
  return metrics[1]

In [ ]:
# Roulette wheel selection method
def selection(population_fitness):
  total = sum(population_fitness)
  percentage = [round((x/total) * 100) for x in population_fitness]
  selection_wheel = []
  for pop_index,num in enumerate(percentage):
    selection_wheel.extend([pop_index]*num)
  parent1_ind = choice(selection_wheel)
  parent2_ind = choice(selection_wheel)
  return [parent1_ind, parent2_ind]

In [ ]:
def crossover_vgg16(parent1, parent2):
  child1 = {}
  child2 = {}

  child1["frozen"] = choice([parent1["frozen"], parent2["frozen"]])
  child2["frozen"] = choice([parent1["frozen"], parent2["frozen"]])

  child1["f1"] = choice([parent1["f1"], parent2["f1"]])
  child2["f1"] = choice([parent1["f1"], parent2["f1"]])

  child1["a1"] = parent2["a1"]
  child2["a1"] = parent1["a1"]

  child1["d1"] = parent1["d1"]
  child2["d1"] = parent2["d1"]

  child1["op"] = parent1["op"]
  child2["op"] = parent2["op"]

  child1["ep"] = parent1["ep"]
  child2["ep"] = parent2["ep"]
  return [child1, child2]

In [ ]:
def mutation_vgg16(chromosome):
  flag = randint(0,40)
  if flag <= 20:
    chromosome["ep"] += randint(0, 10)
  return chromosome

In [ ]:
generations = 5
threshold = 90
num_pop = 5

population = generate_population_vgg16(num_pop)

for generation in range(generations):

  population_fitness = []
  for chromosome in population:
    frozen = chromosome["frozen"]
    f1 = chromosome["f1"]
    a1 = chromosome["a1"]
    d1 = chromosome["d1"]
    op = chromosome["op"]
    ep = chromosome["ep"]

    try:
      model = VGG16_model(frozen, f1, a1, d1, op, ep)
      acc = fitness_evaluation(model)
      print("Parameters: ", chromosome)
      print("Accuracy: ", round(acc,5))
    except:
      acc=0
      print("Parameters: ", chromosome)
      print("Invalid parameters - Build fail")

    population_fitness.append(acc)

  parents_ind = selection(population_fitness)
  parent1 = population[parents_ind[0]]
  parent2 = population[parents_ind[1]]

  children = crossover_vgg16(parent1, parent2)
  child1 = mutation_vgg16(children[0])
  child2 = mutation_vgg16(children[1])

  population.append(child1)
  population.append(child2)

  print("Generation ", generation+1," Outcome: ")
  if max(population_fitness) >= threshold:
    print("Obtained desired accuracy: ", max(population_fitness))
    break
  else:
    print("Maximum accuracy in generation {} : {}".format(generation+1, max(population_fitness)))

  first_min = min(population_fitness)
  first_min_ind = population_fitness.index(first_min)
  population.remove(population[first_min_ind])
  second_min = min(population_fitness)
  second_min_ind = population_fitness.index(second_min)
  population.remove(population[second_min_ind])

In [ ]:
# VGG 19

from keras.applications.vgg19 import VGG19

In [ ]:
#Hyperparameters
# frozen: Frozen layers
# f1: Number of units in dense layer
# a1: Activation functions
# d1: Drop out
# op: Optimizer
# ep: Number of epochs

#Define VGG19 Model
def VGG19_model(frozen, f1, a1, d1, op, ep):
  model_fine_tune = VGG19(include_top=False, weights='imagenet', input_shape=input_shape)
  for layer in model_fine_tune.layers[:frozen]:
      layer.trainable = False
  for layer in model_fine_tune.layers[frozen:]:
      layer.trainable = True
  model = GlobalAveragePooling2D()(model_fine_tune.output)
  model=Dense(units=f1,activation=a1)(model)
  model=Dropout(d1)(model)
  model = Dense(num_class, activation='softmax')(model)
  model = Model(model_fine_tune.input, model, name='vgg')
  model.compile(loss='categorical_crossentropy',optimizer=op,metrics=['accuracy'])
  es = EarlyStopping(monitor="val_accuracy", patience = 4)
  model.fit(train_generator,steps_per_epoch=len(train_generator),epochs=ep,validation_data=validation_generator,
            validation_steps=len(validation_generator), callbacks=[es], verbose=0)
  return model

In [ ]:
from random import choice
from random import uniform
from numpy.random import randint

In [ ]:
def initialization_vgg19():
  parameters = {}
  frozen = choice(range(10, 25, 1))
  parameters["frozen"] = frozen
  f1 = choice([128, 256])
  parameters["f1"] = f1
  a1 = choice(["relu", "selu", "elu"])
  parameters["a1"] = a1
  d1 = round(uniform(0.1, 0.6), 1)
  parameters["d1"] = d1
  op = choice(["adam", "nadam", "adamax", "adagrad"])
  parameters["op"] = op
  ep = randint(10, 25)
  parameters["ep"] = ep
  return parameters

In [ ]:
def generate_population_vgg19(n):
  population = []
  for i in range(n):
    chromosome = initialization_vgg19()
    population.append(chromosome)
  return population

In [ ]:
# Roulette wheel selection method
def selection(population_fitness):
  total = sum(population_fitness)
  percentage = [round((x/total) * 100) for x in population_fitness]
  selection_wheel = []
  for pop_index,num in enumerate(percentage):
    selection_wheel.extend([pop_index]*num)
  parent1_ind = choice(selection_wheel)
  parent2_ind = choice(selection_wheel)
  return [parent1_ind, parent2_ind]

In [ ]:
def crossover_vgg19(parent1, parent2):
  child1 = {}
  child2 = {}

  child1["frozen"] = choice([parent1["frozen"], parent2["frozen"]])
  child2["frozen"] = choice([parent1["frozen"], parent2["frozen"]])

  child1["f1"] = choice([parent1["f1"], parent2["f1"]])
  child2["f1"] = choice([parent1["f1"], parent2["f1"]])

  child1["a1"] = parent2["a1"]
  child2["a1"] = parent1["a1"]

  child1["d1"] = parent1["d1"]
  child2["d1"] = parent2["d1"]

  child1["op"] = parent1["op"]
  child2["op"] = parent2["op"]

  child1["ep"] = parent1["ep"]
  child2["ep"] = parent2["ep"]
  return [child1, child2]

In [ ]:
def mutation_vgg19(chromosome):
  flag = randint(0,40)
  if flag <= 20:
    chromosome["ep"] += randint(0, 10)
  return chromosome

In [ ]:
generations = 5
threshold = 90
num_pop = 3

population = generate_population_vgg19(num_pop)

for generation in range(generations):

  population_fitness = []
  for chromosome in population:
    frozen = chromosome["frozen"]
    f1 = chromosome["f1"]
    a1 = chromosome["a1"]
    d1 = chromosome["d1"]
    op = chromosome["op"]
    ep = chromosome["ep"]

    try:
      model = VGG19_model(frozen, f1, a1, d1, op, ep)
      acc = fitness_evaluation(model)
      print("Parameters: ", chromosome)
      print("Accuracy: ", round(acc,5))
    except:
      acc=0
      print("Parameters: ", chromosome)
      print("Invalid parameters - Build fail")

    population_fitness.append(acc)

  parents_ind = selection(population_fitness)
  parent1 = population[parents_ind[0]]
  parent2 = population[parents_ind[1]]

  children = crossover_vgg19(parent1, parent2)
  child1 = mutation_vgg19(children[0])
  child2 = mutation_vgg19(children[1])

  population.append(child1)
  population.append(child2)

  print("Generation ", generation+1," Outcome: ")
  if max(population_fitness) >= threshold:
    print("Obtained desired accuracy: ", max(population_fitness))
    break
  else:
    print("Maximum accuracy in generation {} : {}".format(generation+1, max(population_fitness)))

  first_min = min(population_fitness)
  first_min_ind = population_fitness.index(first_min)
  population.remove(population[first_min_ind])
  second_min = min(population_fitness)
  second_min_ind = population_fitness.index(second_min)
  population.remove(population[second_min_ind])

In [ ]:
# INCEPTION

from keras.applications.inception_v3 import InceptionV3

In [ ]:
#Hyperparameters
# frozen: Frozen layers
# f1: Number of units in dense layer
# a1: Activation functions
# d1: Drop out
# op: Optimizer
# ep: Number of epochs

#Define Inception Model
def Inception_model(frozen, f1, a1, d1, op, ep):
  model_fine_tune = InceptionV3(include_top=False, weights='imagenet', input_shape=input_shape)
  for layer in model_fine_tune.layers[:frozen]:
      layer.trainable = False
  for layer in model_fine_tune.layers[frozen:]:
      layer.trainable = True
  model = GlobalAveragePooling2D()(model_fine_tune.output)
  model=Dense(units=f1,activation=a1)(model)
  model=Dropout(d1)(model)
  model = Dense(num_class, activation='softmax')(model)
  model = Model(model_fine_tune.input, model, name='resnet')
  model.compile(loss='categorical_crossentropy',optimizer=op,metrics=['accuracy'])
  es = EarlyStopping(monitor="val_accuracy", patience = 4)
  model.fit(train_generator,steps_per_epoch=len(train_generator),epochs=ep,validation_data=validation_generator,
            validation_steps=len(validation_generator), callbacks=[es], verbose=0)
  return model

In [ ]:
from random import choice
from random import uniform
from numpy.random import randint

In [ ]:
def initialization_inception():
  parameters = {}
  frozen = choice(range(30, 50, 1))
  parameters["frozen"] = frozen
  f1 = choice([128, 256])
  parameters["f1"] = f1
  a1 = choice(["relu", "selu", "elu"])
  parameters["a1"] = a1
  d1 = round(uniform(0.1, 0.6), 1)
  parameters["d1"] = d1
  op = choice(["adam", "nadam", "adamax", "adagrad"])
  parameters["op"] = op
  ep = randint(10, 25)
  parameters["ep"] = ep
  return parameters

In [ ]:
def generate_population_inception(n):
  population = []
  for i in range(n):
    chromosome = initialization_inception()
    population.append(chromosome)
  return population

In [ ]:
# Fitness evaluation metric: Classification Accuracy
def fitness_evaluation(model):
  metrics = model.evaluate(validation_generator)
  return metrics[1]

In [ ]:
# Roulette wheel selection method
def selection(population_fitness):
  total = sum(population_fitness)
  percentage = [round((x/total) * 100) for x in population_fitness]
  selection_wheel = []
  for pop_index,num in enumerate(percentage):
    selection_wheel.extend([pop_index]*num)
  parent1_ind = choice(selection_wheel)
  parent2_ind = choice(selection_wheel)
  return [parent1_ind, parent2_ind]

In [ ]:
def crossover_inception(parent1, parent2):
  child1 = {}
  child2 = {}

  child1["frozen"] = choice([parent1["frozen"], parent2["frozen"]])
  child2["frozen"] = choice([parent1["frozen"], parent2["frozen"]])

  child1["f1"] = choice([parent1["f1"], parent2["f1"]])
  child2["f1"] = choice([parent1["f1"], parent2["f1"]])

  child1["a1"] = parent2["a1"]
  child2["a1"] = parent1["a1"]

  child1["d1"] = parent1["d1"]
  child2["d1"] = parent2["d1"]

  child1["op"] = parent1["op"]
  child2["op"] = parent2["op"]

  child1["ep"] = parent1["ep"]
  child2["ep"] = parent2["ep"]
  return [child1, child2]

In [ ]:
def mutation_inception(chromosome):
  flag = randint(0,40)
  if flag <= 20:
    chromosome["ep"] += randint(0, 10)
  return chromosome

In [ ]:
generations = 5
threshold = 90
num_pop = 3

population = generate_population_inception(num_pop)

for generation in range(generations):

  population_fitness = []
  for chromosome in population:
    frozen = chromosome["frozen"]
    f1 = chromosome["f1"]
    a1 = chromosome["a1"]
    d1 = chromosome["d1"]
    op = chromosome["op"]
    ep = chromosome["ep"]

    try:
      model = Inception_model(frozen, f1, a1, d1, op, ep)
      acc = fitness_evaluation(model)
      print("Parameters: ", chromosome)
      print("Accuracy: ", round(acc,5))
    except:
      acc=0
      print("Parameters: ", chromosome)
      print("Invalid parameters - Build fail")

    population_fitness.append(acc)

  parents_ind = selection(population_fitness)
  parent1 = population[parents_ind[0]]
  parent2 = population[parents_ind[1]]

  children = crossover_inception(parent1, parent2)
  child1 = mutation_inception(children[0])
  child2 = mutation_inception(children[1])

  population.append(child1)
  population.append(child2)

  print("Generation ", generation+1," Outcome: ")
  if max(population_fitness) >= threshold:
    print("Obtained desired accuracy: ", max(population_fitness))
    break
  else:
    print("Maximum accuracy in generation {} : {}".format(generation+1, max(population_fitness)))

  first_min = min(population_fitness)
  first_min_ind = population_fitness.index(first_min)
  population.remove(population[first_min_ind])
  second_min = min(population_fitness)
  second_min_ind = population_fitness.index(second_min)
  population.remove(population[second_min_ind])

In [ ]:
from keras.applications.inception_resnet_v2 import InceptionResNetV2

In [ ]:
#Hyperparameters
# frozen: Frozen layers
# f1: Number of units in dense layer
# a1: Activation functions
# d1: Drop out
# op: Optimizer
# ep: Number of epochs

#Define InceptionResnet Model
def InceptionRes_model(frozen, f1, a1, d1, op, ep):
  model_fine_tune = InceptionResNetV2(include_top=False, weights='imagenet', input_shape=input_shape)
  for layer in model_fine_tune.layers[:frozen]:
      layer.trainable = False
  for layer in model_fine_tune.layers[frozen:]:
      layer.trainable = True
  model = GlobalAveragePooling2D()(model_fine_tune.output)
  model=Dense(units=f1,activation=a1)(model)
  model=Dropout(d1)(model)
  model = Dense(num_class, activation='softmax')(model)
  model = Model(model_fine_tune.input, model, name='resnet')
  model.compile(loss='categorical_crossentropy',optimizer=op,metrics=['accuracy'])
  es = EarlyStopping(monitor="val_accuracy", patience = 4)
  model.fit(train_generator,steps_per_epoch=len(train_generator),epochs=ep,validation_data=validation_generator,
            validation_steps=len(validation_generator), callbacks=[es], verbose=0)
  return model

In [ ]:
from random import choice
from random import uniform
from numpy.random import randint

In [ ]:
def initialization_IncepRes():
  parameters = {}
  frozen = choice(range(400, 600, 1))
  parameters["frozen"] = frozen
  f1 = choice([128, 256])
  parameters["f1"] = f1
  a1 = choice(["relu", "selu", "elu"])
  parameters["a1"] = a1
  d1 = round(uniform(0.1, 0.6), 1)
  parameters["d1"] = d1
  op = choice(["adam", "nadam", "adamax", "adagrad"])
  parameters["op"] = op
  ep = randint(10, 25)
  parameters["ep"] = ep
  return parameters

In [ ]:
def generate_population_IncepRes(n):
  population = []
  for i in range(n):
    chromosome = initialization_IncepRes()
    population.append(chromosome)
  return population

In [ ]:
# Fitness evaluation metric: Classification Accuracy
def fitness_evaluation(model):
  metrics = model.evaluate(validation_generator)
  return metrics[1]

In [ ]:
# Roulette wheel selection method
def selection(population_fitness):
  total = sum(population_fitness)
  percentage = [round((x/total) * 100) for x in population_fitness]
  selection_wheel = []
  for pop_index,num in enumerate(percentage):
    selection_wheel.extend([pop_index]*num)
  parent1_ind = choice(selection_wheel)
  parent2_ind = choice(selection_wheel)
  return [parent1_ind, parent2_ind]

In [ ]:
def crossover_IncepRes(parent1, parent2):
  child1 = {}
  child2 = {}

  child1["frozen"] = choice([parent1["frozen"], parent2["frozen"]])
  child2["frozen"] = choice([parent1["frozen"], parent2["frozen"]])

  child1["f1"] = choice([parent1["f1"], parent2["f1"]])
  child2["f1"] = choice([parent1["f1"], parent2["f1"]])

  child1["a1"] = parent2["a1"]
  child2["a1"] = parent1["a1"]

  child1["d1"] = parent1["d1"]
  child2["d1"] = parent2["d1"]

  child1["op"] = parent1["op"]
  child2["op"] = parent2["op"]

  child1["ep"] = parent1["ep"]
  child2["ep"] = parent2["ep"]
  return [child1, child2]

In [ ]:
def mutation_IncepRes(chromosome):
  flag = randint(0,40)
  if flag <= 20:
    chromosome["ep"] += randint(0, 10)
  return chromosome

In [ ]:
generations = 5
threshold = 90
num_pop = 3

population = generate_population_IncepRes(num_pop)

for generation in range(generations):

  population_fitness = []
  for chromosome in population:
    frozen = chromosome["frozen"]
    f1 = chromosome["f1"]
    a1 = chromosome["a1"]
    d1 = chromosome["d1"]
    op = chromosome["op"]
    ep = chromosome["ep"]

    try:
      model = InceptionRes_model(frozen, f1, a1, d1, op, ep)
      acc = fitness_evaluation(model)
      print("Parameters: ", chromosome)
      print("Accuracy: ", round(acc,5))
    except:
      acc=0
      print("Parameters: ", chromosome)
      print("Invalid parameters - Build fail")

    population_fitness.append(acc)

  parents_ind = selection(population_fitness)
  parent1 = population[parents_ind[0]]
  parent2 = population[parents_ind[1]]

  children = crossover_IncepRes(parent1, parent2)
  child1 = mutation_IncepRes(children[0])
  child2 = mutation_IncepRes(children[1])

  population.append(child1)
  population.append(child2)

  print("Generation ", generation+1," Outcome: ")
  if max(population_fitness) >= threshold:
    print("Obtained desired accuracy: ", max(population_fitness))
    break
  else:
    print("Maximum accuracy in generation {} : {}".format(generation+1, max(population_fitness)))

  first_min = min(population_fitness)
  first_min_ind = population_fitness.index(first_min)
  population.remove(population[first_min_ind])
  second_min = min(population_fitness)
  second_min_ind = population_fitness.index(second_min)
  population.remove(population[second_min_ind])

In [ ]:
from keras.applications.efficientnet import EfficientNetB7

In [ ]:
#Hyperparameters
# frozen: Frozen layers
# f1: Number of units in dense layer
# a1: Activation functions
# d1: Drop out
# op: Optimizer
# ep: Number of epochs

#Define EfficientNetB7 Model
def EfficientNetB7_model(frozen, f1, a1, d1, op, ep):
  model_fine_tune = EfficientNetB7(include_top=False, weights='imagenet', input_shape=input_shape)
  for layer in model_fine_tune.layers[:frozen]:
      layer.trainable = False
  for layer in model_fine_tune.layers[frozen:]:
      layer.trainable = True
  model = GlobalAveragePooling2D()(model_fine_tune.output)
  model=Dense(units=f1,activation=a1)(model)
  model=Dropout(d1)(model)
  model = Dense(num_class, activation='softmax')(model)
  model = Model(model_fine_tune.input, model, name='efficientnetb7')
  model.compile(loss='categorical_crossentropy',optimizer=op,metrics=['accuracy'])
  es = EarlyStopping(monitor="val_accuracy", patience = 4)
  model.fit(train_generator,steps_per_epoch=len(train_generator),epochs=ep,validation_data=validation_generator,
            validation_steps=len(validation_generator), callbacks=[es], verbose=0)
  return model

In [ ]:
from random import choice
from random import uniform
from numpy.random import randint

In [ ]:
def initialization_EfficientNetB7():
  parameters = {}
  frozen = choice(range(270, 350, 1))
  parameters["frozen"] = frozen
  f1 = choice([128, 256])
  parameters["f1"] = f1
  a1 = choice(["relu", "selu", "elu"])
  parameters["a1"] = a1
  d1 = round(uniform(0.1, 0.6), 1)
  parameters["d1"] = d1
  op = choice(["adam", "nadam", "adamax", "adagrad"])
  parameters["op"] = op
  ep = randint(10, 25)
  parameters["ep"] = ep
  return parameters

In [ ]:
def generate_population_EfficientNetB7(n):
  population = []
  for i in range(n):
    chromosome = initialization_EfficientNetB7()
    population.append(chromosome)
  return population

In [ ]:
# Fitness evaluation metric: Classification Accuracy
def fitness_evaluation(model):
  metrics = model.evaluate(validation_generator)
  return metrics[1]

In [ ]:
# Roulette wheel selection method
def selection(population_fitness):
  total = sum(population_fitness)
  percentage = [round((x/total) * 100) for x in population_fitness]
  selection_wheel = []
  for pop_index,num in enumerate(percentage):
    selection_wheel.extend([pop_index]*num)
  parent1_ind = choice(selection_wheel)
  parent2_ind = choice(selection_wheel)
  return [parent1_ind, parent2_ind]

In [ ]:
def crossover_EfficientNetB7(parent1, parent2):
  child1 = {}
  child2 = {}

  child1["frozen"] = choice([parent1["frozen"], parent2["frozen"]])
  child2["frozen"] = choice([parent1["frozen"], parent2["frozen"]])

  child1["f1"] = choice([parent1["f1"], parent2["f1"]])
  child2["f1"] = choice([parent1["f1"], parent2["f1"]])

  child1["a1"] = parent2["a1"]
  child2["a1"] = parent1["a1"]

  child1["d1"] = parent1["d1"]
  child2["d1"] = parent2["d1"]

  child1["op"] = parent1["op"]
  child2["op"] = parent2["op"]

  child1["ep"] = parent1["ep"]
  child2["ep"] = parent2["ep"]
  return [child1, child2]

In [ ]:
def mutation_EfficientNetB7(chromosome):
  flag = randint(0,40)
  if flag <= 20:
    chromosome["ep"] += randint(0, 10)
  return chromosome

In [ ]:
generations = 5
threshold = 90
num_pop = 3

population = generate_population_EfficientNetB7(num_pop)

for generation in range(generations):

  population_fitness = []
  for chromosome in population:
    frozen = chromosome["frozen"]
    f1 = chromosome["f1"]
    a1 = chromosome["a1"]
    d1 = chromosome["d1"]
    op = chromosome["op"]
    ep = chromosome["ep"]

    try:
      model = EfficientNetB7_model(frozen, f1, a1, d1, op, ep)
      acc = fitness_evaluation(model)
      print("Parameters: ", chromosome)
      print("Accuracy: ", round(acc,5))
    except:
      acc=0
      print("Parameters: ", chromosome)
      print("Invalid parameters - Build fail")

    population_fitness.append(acc)

  parents_ind = selection(population_fitness)
  parent1 = population[parents_ind[0]]
  parent2 = population[parents_ind[1]]

  children = crossover_EfficientNetB7(parent1, parent2)
  child1 = mutation_EfficientNetB7(children[0])
  child2 = mutation_EfficientNetB7(children[1])

  population.append(child1)
  population.append(child2)

  print("Generation ", generation+1," Outcome: ")
  if max(population_fitness) >= threshold:
    print("Obtained desired accuracy: ", max(population_fitness))
    break
  else:
    print("Maximum accuracy in generation {} : {}".format(generation+1, max(population_fitness)))

  first_min = min(population_fitness)
  first_min_ind = population_fitness.index(first_min)
  population.remove(population[first_min_ind])
  second_min = min(population_fitness)
  second_min_ind = population_fitness.index(second_min)
  population.remove(population[second_min_ind])

In [ ]:
from keras.applications.efficientnet_v2 import EfficientNetV2L

In [ ]:
#Hyperparameters
# frozen: Frozen layers
# f1: Number of units in dense layer
# a1: Activation functions
# d1: Drop out
# op: Optimizer
# ep: Number of epochs

#Define EfficientNetV2L Model
def EfficientNetV2L_model(frozen, f1, a1, d1, op, ep):
  model_fine_tune = EfficientNetV2L(include_top=False, weights='imagenet', input_shape=input_shape)
  for layer in model_fine_tune.layers[:frozen]:
      layer.trainable = False
  for layer in model_fine_tune.layers[frozen:]:
      layer.trainable = True
  model = GlobalAveragePooling2D()(model_fine_tune.output)
  model=Dense(units=f1,activation=a1)(model)
  model=Dropout(d1)(model)
  model = Dense(num_class, activation='softmax')(model)
  model = Model(model_fine_tune.input, model, name='efficientnetv2-l')
  model.compile(loss='categorical_crossentropy',optimizer=op,metrics=['accuracy'])
  es = EarlyStopping(monitor="val_accuracy", patience = 4)
  model.fit(train_generator,steps_per_epoch=len(train_generator),epochs=ep,validation_data=validation_generator,
            validation_steps=len(validation_generator), callbacks=[es], verbose=0)
  return model

In [ ]:
from random import choice
from random import uniform
from numpy.random import randint

In [ ]:
def initialization_EfficientNetV2L():
  parameters = {}
  frozen = choice(range(380, 430, 1))
  parameters["frozen"] = frozen
  f1 = choice([128, 256])
  parameters["f1"] = f1
  a1 = choice(["relu", "selu", "elu"])
  parameters["a1"] = a1
  d1 = round(uniform(0.1, 0.6), 1)
  parameters["d1"] = d1
  op = choice(["adam", "nadam", "adamax", "adagrad"])
  parameters["op"] = op
  ep = randint(10, 25)
  parameters["ep"] = ep
  return parameters

In [ ]:
def generate_population_EfficientNetV2L(n):
  population = []
  for i in range(n):
    chromosome = initialization_EfficientNetV2L()
    population.append(chromosome)
  return population

In [ ]:
# Fitness evaluation metric: Classification Accuracy
def fitness_evaluation(model):
  metrics = model.evaluate(validation_generator)
  return metrics[1]

In [ ]:
# Roulette wheel selection method
def selection(population_fitness):
  total = sum(population_fitness)
  percentage = [round((x/total) * 100) for x in population_fitness]
  selection_wheel = []
  for pop_index,num in enumerate(percentage):
    selection_wheel.extend([pop_index]*num)
  parent1_ind = choice(selection_wheel)
  parent2_ind = choice(selection_wheel)
  return [parent1_ind, parent2_ind]

In [ ]:
def crossover_EfficientNetV2L(parent1, parent2):
  child1 = {}
  child2 = {}

  child1["frozen"] = choice([parent1["frozen"], parent2["frozen"]])
  child2["frozen"] = choice([parent1["frozen"], parent2["frozen"]])

  child1["f1"] = choice([parent1["f1"], parent2["f1"]])
  child2["f1"] = choice([parent1["f1"], parent2["f1"]])

  child1["a1"] = parent2["a1"]
  child2["a1"] = parent1["a1"]

  child1["d1"] = parent1["d1"]
  child2["d1"] = parent2["d1"]

  child1["op"] = parent1["op"]
  child2["op"] = parent2["op"]

  child1["ep"] = parent1["ep"]
  child2["ep"] = parent2["ep"]
  return [child1, child2]

In [ ]:
def mutation_EfficientNetV2L(chromosome):
  flag = randint(0,40)
  if flag <= 20:
    chromosome["ep"] += randint(0, 10)
  return chromosome

In [ ]:

generations = 5
threshold = 90
num_pop = 3

population = generate_population_EfficientNetB7(num_pop)

for generation in range(generations):

  population_fitness = []
  for chromosome in population:
    frozen = chromosome["frozen"]
    f1 = chromosome["f1"]
    a1 = chromosome["a1"]
    d1 = chromosome["d1"]
    op = chromosome["op"]
    ep = chromosome["ep"]

    try:
      model = EfficientNetV2L_model(frozen, f1, a1, d1, op, ep)
      acc = fitness_evaluation(model)
      print("Parameters: ", chromosome)
      print("Accuracy: ", round(acc,5))
    except:
      acc=0
      print("Parameters: ", chromosome)
      print("Invalid parameters - Build fail")

    population_fitness.append(acc)

  parents_ind = selection(population_fitness)
  parent1 = population[parents_ind[0]]
  parent2 = population[parents_ind[1]]

  children = crossover_EfficientNetV2L(parent1, parent2)
  child1 = mutation_EfficientNetV2L(children[0])
  child2 = mutation_EfficientNetV2L(children[1])

  population.append(child1)
  population.append(child2)

  print("Generation ", generation+1," Outcome: ")
  if max(population_fitness) >= threshold:
    print("Obtained desired accuracy: ", max(population_fitness))
    break
  else:
    print("Maximum accuracy in generation {} : {}".format(generation+1, max(population_fitness)))

  first_min = min(population_fitness)
  first_min_ind = population_fitness.index(first_min)
  population.remove(population[first_min_ind])
  second_min = min(population_fitness)
  second_min_ind = population_fitness.index(second_min)
  population.remove(population[second_min_ind])